In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
import json
import wasp
from tqdm.autonotebook import tqdm

C:\Python37\lib\site-packages\ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
with open(wasp.get_data_path("item_pool", "item_pool_supp.json"), "rb") as fin:
    items = json.load(fin)

In [41]:
import requests
import requests_cache
requests_cache.install_cache()

In [42]:
from itertools import chain

pooling_func = lambda x: x["question"] + list(chain.from_iterable(x["options"]))
tokens = list(set(chain.from_iterable(map(pooling_func, items))))

In [53]:
import time
def query_conceptnet(token):
    url_templ = "http://api.conceptnet.io/c/zh/{query}?limit=100"
    url = url_templ.format(query=token)
    resp = requests.get(url)
    data = []    
    while resp and resp.status_code == 200:        
        # print(f"request {url}")
        json = resp.json()
        data.append(json)
        nextPage = json.get("view", {}).get("nextPage")
        if nextPage:
            url = "http://api.conceptnet.io" + nextPage
            if hasattr(resp, "from_cache") and not resp.from_cache:
                time.sleep(0.8)
            resp = requests.get(url)
        else:
            resp = None
    return data

In [54]:
# retrieve conceptnet data
concept_data = {}
for tok in tqdm(tokens):
    data = query_conceptnet(tok)
    concept_data[tok] = data
    time.sleep(0.8)

In [55]:
import pickle
wasp.install_data_dir("sem_graph")
with open(wasp.get_data_path("sem_graph", "concept_net.pkl"), "wb") as fout:
    pickle.dump(concept_data, fout)